In [1]:
from sklearn.linear_model import ElasticNet, LinearRegression as lr, LogisticRegression
from sklearn.ensemble import GradientBoostingRegressor as gbr, RandomForestRegressor as rfr
from xgboost import XGBRegressor

/home/paul/env/py35/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# Useful if you are debugging the function inside another .py script
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd

houses_train = pd.read_csv('../Data/encoded_houses_train.csv')
houses_test = pd.read_csv('../Data/encoded_houses_test.csv')

In [4]:
X_train = houses_train.loc[:, houses_train.columns != "SalePrice"].values # convert to np.array
y_train = houses_train.loc[:, houses_train.columns == "SalePrice"].values.reshape(-1, )

X_test = houses_test.loc[:, houses_train.columns != "SalePrice"].values # convert to np.array

In [73]:
from stacking import stacking_regression, append_meta_features
from sklearn.metrics import mean_squared_error
import numpy as np

In [6]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(np.log(y), np.log(y_pred)))

In [10]:
from  sklearn import preprocessing
y_quant = pd.qcut(y_train, 5, labels=range(1,6))
le = preprocessing.LabelEncoder()
le.fit(y_quant)
y_quant = le.transform(y_quant)

In [17]:
from sklearn.model_selection import train_test_split
import sklearn.model_selection as ms
from sklearn import metrics

seed = 10
test_ratio = 0.2

X_pr_train, X_pr_test, y_pr_train, y_pr_test = train_test_split(X_train, y_quant, test_size=test_ratio, random_state=seed)

In [16]:
param_grid = { "C" : 10**np.arange(0, 4, 0.4)}
logitCV = LogisticRegression()
logitCV.set_params(random_state=67)
grid_search_logit = ms.GridSearchCV(logitCV, param_grid, cv=5, n_jobs=-1, verbose=10, scoring='accuracy')
%time grid_search_logit.fit(X_train, y_quant)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] C=1.0 ...........................................................
[CV] C=1.0 ...........................................................
[CV] C=1.0 ...........................................................
[CV] C=1.0 ...........................................................
[CV] C=1.0 ...........................................................
[CV] C=2.51188643151 .................................................
[CV] C=2.51188643151 .................................................
[CV] C=2.51188643151 .................................................
[CV] .................. C=1.0, score=0.6027397260273972, total=   1.8s
[CV] C=2.51188643151 .................................................
[CV] .................. C=1.0, score=0.6143344709897611, total=   1.8s
[CV] C=2.51188643151 .................................................
[CV] .................. C=1.0, score=0.6143344709897611, total=   2.0s
[CV] C=6.3095734

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.9s


[CV] ........ C=2.51188643151, score=0.6075085324232082, total=   2.7s
[CV] C=6.3095734448 ..................................................
[CV] ........ C=2.51188643151, score=0.6075085324232082, total=   2.8s
[CV] C=6.3095734448 ..................................................
[CV] ......... C=2.51188643151, score=0.678082191780822, total=   3.0s
[CV] C=15.8489319246 .................................................
[CV] ........ C=2.51188643151, score=0.6164383561643836, total=   2.9s
[CV] C=15.8489319246 .................................................
[CV] ........ C=2.51188643151, score=0.5896551724137931, total=   2.9s
[CV] C=15.8489319246 .................................................


[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    4.8s


[CV] ......... C=6.3095734448, score=0.6075085324232082, total=   3.7s
[CV] C=15.8489319246 .................................................
[CV] ......... C=6.3095734448, score=0.6177474402730375, total=   3.8s
[CV] ......... C=6.3095734448, score=0.6643835616438356, total=   3.8s
[CV] C=15.8489319246 .................................................
[CV] C=39.8107170553 .................................................
[CV] ......... C=6.3095734448, score=0.5827586206896552, total=   3.5s
[CV] C=39.8107170553 .................................................
[CV] ......... C=6.3095734448, score=0.6164383561643836, total=   3.7s
[CV] C=39.8107170553 .................................................
[CV] ........ C=15.8489319246, score=0.6040955631399317, total=   4.5s
[CV] C=39.8107170553 .................................................


...

[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:  1.1min remaining:    4.2s


[CV] ........ C=3981.07170553, score=0.5836177474402731, total=  29.0s
[CV] ........ C=3981.07170553, score=0.5862068965517241, total=  18.0s
[CV] ........ C=3981.07170553, score=0.5993150684931506, total=  18.7s


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.2min finished


CPU times: user 908 ms, sys: 64 ms, total: 972 ms
Wall time: 1min 10s


GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=67, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'C': array([  1.00000e+00,   2.51189e+00,   6.30957e+00,   1.58489e+01,
         3.98107e+01,   1.00000e+02,   2.51189e+02,   6.30957e+02,
         1.58489e+03,   3.98107e+03])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=10)

In [31]:
print(grid_search_logit.best_params_)
print(grid_search_logit.best_score_)

{'C': 1.0}
0.621232876712


In [24]:
from sklearn.ensemble import RandomForestClassifier
tree_quant = RandomForestClassifier(n_estimators=1000, max_features=50, max_depth=25)

param_grid = { "n_estimators"      : [500, 700],
           "max_features"      : [10, 25, 50, 75],
           "max_depth"         : [10, 20, 25, 30]}
tree_quant.set_params(random_state=67)
grid_search_tree = ms.GridSearchCV(tree_quant, param_grid, cv=5, n_jobs=-1, verbose=10, scoring='accuracy')
%time grid_search_tree.fit(X_train, y_quant)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV] max_depth=10, n_estimators=500, max_features=10 .................
[CV] max_depth=10, n_estimators=500, max_features=10 .................
[CV] max_depth=10, n_estimators=500, max_features=10 .................
[CV] max_depth=10, n_estimators=500, max_features=10 .................
[CV] max_depth=10, n_estimators=700, max_features=10 .................
[CV] max_depth=10, n_estimators=500, max_features=10 .................
[CV] max_depth=10, n_estimators=700, max_features=10 .................
[CV] max_depth=10, n_estimators=700, max_features=10 .................
[CV]  max_depth=10, n_estimators=500, max_features=10, score=0.7226027397260274, total=   1.5s
[CV] max_depth=10, n_estimators=700, max_features=10 .................
[CV]  max_depth=10, n_estimators=500, max_features=10, score=0.658703071672355, total=   1.6s
[CV]  max_depth=10, n_estimators=500, max_features=10, score=0.7054794520547946, total=   1.6s
[CV] max_depth=

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.7s


[CV]  max_depth=10, n_estimators=700, max_features=10, score=0.6621160409556314, total=   2.2s
[CV] max_depth=10, n_estimators=500, max_features=25 .................
[CV]  max_depth=10, n_estimators=700, max_features=10, score=0.6928327645051194, total=   2.2s
[CV] max_depth=10, n_estimators=500, max_features=25 .................
[CV]  max_depth=10, n_estimators=700, max_features=10, score=0.7226027397260274, total=   2.2s
[CV] max_depth=10, n_estimators=700, max_features=25 .................
[CV]  max_depth=10, n_estimators=700, max_features=10, score=0.708904109589041, total=   2.2s
[CV] max_depth=10, n_estimators=700, max_features=25 .................
[CV]  max_depth=10, n_estimators=700, max_features=10, score=0.6551724137931034, total=   2.2s
[CV] max_depth=10, n_estimators=700, max_features=25 .................
[CV]  max_depth=10, n_estimators=500, max_features=25, score=0.7098976109215017, total=   2.4s
[CV] max_depth=10, n_estimators=700, max_features=25 .................


[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    4.2s


[CV]  max_depth=10, n_estimators=500, max_features=25, score=0.6860068259385665, total=   2.5s
[CV] max_depth=10, n_estimators=700, max_features=25 .................
[CV]  max_depth=10, n_estimators=500, max_features=25, score=0.726027397260274, total=   2.5s
[CV] max_depth=10, n_estimators=500, max_features=50 .................
[CV]  max_depth=10, n_estimators=500, max_features=25, score=0.7294520547945206, total=   2.4s
[CV] max_depth=10, n_estimators=500, max_features=50 .................
[CV]  max_depth=10, n_estimators=500, max_features=25, score=0.6655172413793103, total=   2.6s
[CV] max_depth=10, n_estimators=500, max_features=50 .................
[CV]  max_depth=10, n_estimators=700, max_features=25, score=0.6791808873720137, total=   3.3s
[CV] max_depth=10, n_estimators=500, max_features=50 .................


...

[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:  1.1min


[CV]  max_depth=30, n_estimators=500, max_features=25, score=0.6928327645051194, total=   2.6s
[CV] max_depth=30, n_estimators=700, max_features=25 .................
[CV]  max_depth=30, n_estimators=500, max_features=25, score=0.6928327645051194, total=   2.7s
[CV] max_depth=30, n_estimators=700, max_features=25 .................
[CV]  max_depth=30, n_estimators=500, max_features=25, score=0.7226027397260274, total=   2.7s
[CV] max_depth=30, n_estimators=500, max_features=50 .................
[CV]  max_depth=30, n_estimators=500, max_features=25, score=0.7328767123287672, total=   2.7s
[CV] max_depth=30, n_estimators=500, max_features=50 .................
[CV]  max_depth=30, n_estimators=500, max_features=25, score=0.6689655172413793, total=   2.7s
[CV] max_depth=30, n_estimators=500, max_features=50 .................
[CV]  max_depth=30, n_estimators=700, max_features=25, score=0.7030716723549488, total=   3.6s
[CV] max_depth=30, n_estimators=500, max_features=50 .................
[CV]

[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:  1.4min finished


CPU times: user 4.96 s, sys: 124 ms, total: 5.08 s
Wall time: 1min 29s


GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=25, max_features=50, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=67, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [500, 700], 'max_depth': [10, 20, 25, 30], 'max_features': [10, 25, 50, 75]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=10)

In [29]:
print(grid_search_tree.best_params_)
print(grid_search_tree.best_score_)

{'max_depth': 20, 'n_estimators': 500, 'max_features': 75}
0.712328767123


In [28]:
from sklearn.ensemble import GradientBoostingClassifier

gbm_quant = GradientBoostingClassifier(random_state=0, learning_rate = 0.01, max_features='sqrt',
        min_samples_leaf=15, min_samples_split=10, 
        n_estimators=1000, max_depth=3)

param_grid = { "n_estimators"      : [500, 1000],
           "max_features"      : [10, 25, 50, 75],
           "max_depth"         : [5, 10, 20, 25, 30]}
gbm_quant.set_params(random_state=67)
grid_search_gbm = ms.GridSearchCV(gbm_quant, param_grid, cv=5, n_jobs=-1, verbose=10, scoring='accuracy')
%time grid_search_gbm.fit(X_train, y_quant)

Fitting 5 folds for each of 40 candidates, totalling 200 fits
[CV] max_depth=5, n_estimators=500, max_features=10 ..................
[CV] max_depth=5, n_estimators=500, max_features=10 ..................
[CV] max_depth=5, n_estimators=500, max_features=10 ..................
[CV] max_depth=5, n_estimators=500, max_features=10 ..................
[CV] max_depth=5, n_estimators=500, max_features=10 ..................
[CV] max_depth=5, n_estimators=1000, max_features=10 .................
[CV] max_depth=5, n_estimators=1000, max_features=10 .................
[CV] max_depth=5, n_estimators=1000, max_features=10 .................
[CV]  max_depth=5, n_estimators=500, max_features=10, score=0.7030716723549488, total=   7.0s
[CV] max_depth=5, n_estimators=1000, max_features=10 .................
[CV]  max_depth=5, n_estimators=500, max_features=10, score=0.7328767123287672, total=   7.1s
[CV] max_depth=5, n_estimators=1000, max_features=10 .................
[CV]  max_depth=5, n_estimators=500, max

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    7.2s


[CV]  max_depth=5, n_estimators=500, max_features=10, score=0.7363013698630136, total=   7.3s
[CV] max_depth=5, n_estimators=500, max_features=25 ..................
[CV]  max_depth=5, n_estimators=500, max_features=10, score=0.6758620689655173, total=   7.3s
[CV] max_depth=5, n_estimators=500, max_features=25 ..................
[CV]  max_depth=5, n_estimators=1000, max_features=10, score=0.6757679180887372, total=  13.7s
[CV] max_depth=5, n_estimators=500, max_features=25 ..................
[CV]  max_depth=5, n_estimators=1000, max_features=10, score=0.7064846416382252, total=  13.8s
[CV] max_depth=5, n_estimators=500, max_features=25 ..................
[CV]  max_depth=5, n_estimators=1000, max_features=10, score=0.6986301369863014, total=  14.0s
[CV] max_depth=5, n_estimators=1000, max_features=25 .................
[CV]  max_depth=5, n_estimators=500, max_features=25, score=0.6860068259385665, total=  10.7s
[CV] max_depth=5, n_estimators=1000, max_features=25 .................


[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   18.2s


[CV]  max_depth=5, n_estimators=500, max_features=25, score=0.7030716723549488, total=  10.9s
[CV] max_depth=5, n_estimators=1000, max_features=25 .................
[CV]  max_depth=5, n_estimators=500, max_features=25, score=0.7397260273972602, total=  11.1s
[CV] max_depth=5, n_estimators=1000, max_features=25 .................
[CV]  max_depth=5, n_estimators=1000, max_features=10, score=0.726027397260274, total=  14.0s
[CV] max_depth=5, n_estimators=1000, max_features=25 .................
[CV]  max_depth=5, n_estimators=1000, max_features=10, score=0.696551724137931, total=  14.1s
[CV] max_depth=5, n_estimators=500, max_features=50 ..................
[CV]  max_depth=5, n_estimators=500, max_features=25, score=0.7328767123287672, total=  10.9s
[CV] max_depth=5, n_estimators=500, max_features=50 ..................
[CV]  max_depth=5, n_estimators=500, max_features=25, score=0.6827586206896552, total=  10.9s
[CV] max_depth=5, n_estimators=500, max_features=50 ..................
[CV]  max_

...

[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed: 14.9min


[CV]  max_depth=30, n_estimators=500, max_features=50, score=0.6896551724137931, total=  38.1s
[CV] max_depth=30, n_estimators=500, max_features=75 .................
[CV]  max_depth=30, n_estimators=500, max_features=75, score=0.6860068259385665, total=  53.0s
[CV] max_depth=30, n_estimators=500, max_features=75 .................
[CV]  max_depth=30, n_estimators=1000, max_features=50, score=0.689419795221843, total= 1.2min
[CV] max_depth=30, n_estimators=500, max_features=75 .................
[CV]  max_depth=30, n_estimators=1000, max_features=50, score=0.7133105802047781, total= 1.3min
[CV] max_depth=30, n_estimators=1000, max_features=75 ................
[CV]  max_depth=30, n_estimators=1000, max_features=50, score=0.7191780821917808, total= 1.3min
[CV] max_depth=30, n_estimators=1000, max_features=75 ................
[CV]  max_depth=30, n_estimators=1000, max_features=50, score=0.726027397260274, total= 1.3min
[CV] max_depth=30, n_estimators=1000, max_features=75 ................
[C

[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed: 17.2min finished


CPU times: user 22.8 s, sys: 380 ms, total: 23.2 s
Wall time: 17min 30s


GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.01, loss='deviance', max_depth=3,
              max_features='sqrt', max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=15, min_samples_split=10,
              min_weight_fraction_leaf=0.0, n_estimators=1000,
              presort='auto', random_state=67, subsample=1.0, verbose=0,
              warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [500, 1000], 'max_depth': [5, 10, 20, 25, 30], 'max_features': [10, 25, 50, 75]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=10)

In [32]:
print(grid_search_gbm.best_params_)
print(grid_search_gbm.best_score_)

{'max_depth': 10, 'n_estimators': 500, 'max_features': 50}
0.714383561644


In [36]:
from sklearn.ensemble import VotingClassifier
vote1 = LogisticRegression(C = 1)
vote2 = RandomForestClassifier(n_estimators=500, max_features=75, max_depth=20)
vote3 = GradientBoostingClassifier(random_state=0, learning_rate = 0.01, max_features=50,
        min_samples_leaf=15, min_samples_split=10, 
        n_estimators=500, max_depth=10)

voter = VotingClassifier(estimators=[('lr', vote1),('rf', vote2),('gbm', vote3)], voting='hard')
score = ms.cross_val_score(voter, X_train, y_quant, scoring='accuracy', cv=10, verbose=5)

[CV]  ................................................................
[CV] ....................... , score=0.6959459459459459, total=  21.6s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   21.6s remaining:    0.0s


[CV] ....................... , score=0.6554054054054054, total=  22.0s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   43.6s remaining:    0.0s


[CV] ....................... , score=0.6891891891891891, total=  21.8s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  1.1min remaining:    0.0s


[CV] ....................... , score=0.7027027027027027, total=  21.6s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  1.4min remaining:    0.0s


[CV] ....................... , score=0.6870748299319728, total=  21.7s
[CV]  ................................................................
[CV] ....................... , score=0.7724137931034483, total=  21.8s
[CV]  ................................................................
[CV] ....................... , score=0.7172413793103448, total=  21.6s
[CV]  ................................................................
[CV] ....................... , score=0.7361111111111112, total=  21.8s
[CV]  ................................................................
[CV] ....................... , score=0.6736111111111112, total=  21.7s
[CV]  ................................................................
[CV] ....................... , score=0.6993006993006993, total=  21.7s


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  3.6min finished


In [54]:
np.mean(score)

0.70289961671119294

In [56]:
voter.fit(X_train, y_quant)

VotingClassifier(estimators=[('lr', LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)), ('rf', RandomFor...         presort='auto', random_state=0, subsample=1.0, verbose=0,
              warm_start=False))],
         flatten_transform=None, n_jobs=1, voting='hard', weights=None)

In [57]:
vote_pred = voter.predict(X_train)
metrics.accuracy_score(vote_pred, y_quant)

1.0

STACKING

In [58]:
models = [
    # linear model, ElasticNet = lasso + ridge
    {'model': ElasticNet(random_state=0, 
              fit_intercept=True, alpha=0.05, l1_ratio=1),
        'X_train': X_train,
        'y_train': y_train,
        'X_test': X_test},
    
    ElasticNet(random_state=0, 
              fit_intercept=True, alpha=0.18069, l1_ratio=0.01),
    
    # deep random forst model
    rfr(random_state=0,
        n_estimators=1000, max_depth=20,  max_features=70),
    
    # conservative gbm model
    gbr(random_state=0, learning_rate = 0.005, max_features='sqrt',
        min_samples_leaf=15, min_samples_split=10, 
        n_estimators=3000, max_depth=3),
    
    # aggressive gbm model
    gbr(random_state = 0, learning_rate = 0.01, max_features='sqrt',
        min_samples_leaf=10, min_samples_split=5, 
        n_estimators = 1000, max_depth = 9),
    ]

meta_model = lr(normalize=True)

In [ ]:
%%time
stacking_features, stacking_prediction = stacking_regression(models,
                                          X_train, y_train, X_test,
                                          transform_target=np.log1p, transform_pred = np.expm1, 
                                          metric=rmsle, verbose=2, n_folds=3)

In [63]:
print(stacking_features.shape)
print(stacking_prediction.shape)
print(y_quant.shape)

(1460, 5)
(1459, 5)
(1460,)


In [82]:
big_features, big_predictions = append_meta_features(stacking_features, stacking_prediction, y_quant.reshape(-1,1))
print(big_features.shape)

(1460, 6)


In [80]:
np.append(stacking_features, y_quant, 1).shape

ValueError: all the input arrays must have same number of dimensions

In [25]:
def averaging_score(x, y):
    return rmsle(x.mean(axis=1), y)

averaging_score(stacking_features, y_train)

0.20193003366588899

One way to evaluate a meta-model: cross_val_score

In [26]:
meta_model = rfr(random_state=0, n_estimators=500, max_depth=5, max_features=3)

In [27]:
score = ms.cross_val_score(estimator=meta_model, X= stacking_features, y=y_train, cv=5, scoring='neg_mean_squared_log_error')
print(np.sqrt(score * -1))
print(np.mean(np.sqrt(score * -1)))

[ 0.09189106  0.11283033  0.10028609  0.08961675  0.09178818]
0.0972824838602


another way -- using gridsearchCV to tune params, and use the crossval score from best param set

In [43]:
param_grid = { "n_estimators"      : [500, 700],
           "max_features"      : range(1, 7),
           "max_depth"         : range(1, 10)}
meta_model.set_params(random_state=67)
grid_search_tree = ms.GridSearchCV(meta_model, param_grid, cv=5, n_jobs=-1, verbose=10, scoring='neg_mean_squared_log_error')
%time grid_search_tree.fit(stacking_features, y_train)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
[CV] max_features=1, max_depth=1, n_estimators=500 ...................
[CV] max_features=1, max_depth=1, n_estimators=500 ...................
[CV] max_features=1, max_depth=1, n_estimators=500 ...................
[CV] max_features=1, max_depth=1, n_estimators=500 ...................
[CV] max_features=1, max_depth=1, n_estimators=700 ...................
[CV] max_features=1, max_depth=1, n_estimators=700 ...................
[CV] max_features=1, max_depth=1, n_estimators=700 ...................
[CV] max_features=1, max_depth=1, n_estimators=500 ...................
[CV]  max_features=1, max_depth=1, n_estimators=500, score=-0.0717085100043705, total=   0.6s
[CV]  max_features=1, max_depth=1, n_estimators=500, score=-0.07823804465865188, total=   0.7s
[CV] max_features=1, max_depth=1, n_estimators=700 ...................
[CV] max_features=1, max_depth=1, n_estimators=700 ...................
[CV]  max_features=1, max_depth=1, n_e

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.7s


[CV]  max_features=1, max_depth=1, n_estimators=700, score=-0.06908893127642346, total=   0.9s
[CV]  max_features=1, max_depth=1, n_estimators=700, score=-0.06410537418742247, total=   0.9s
[CV] max_features=2, max_depth=1, n_estimators=500 ...................
[CV] max_features=2, max_depth=1, n_estimators=500 ...................
[CV]  max_features=1, max_depth=1, n_estimators=700, score=-0.07795334948997333, total=   1.0s
[CV] max_features=2, max_depth=1, n_estimators=700 ...................
[CV]  max_features=2, max_depth=1, n_estimators=500, score=-0.06495272397980129, total=   0.7s
[CV] max_features=2, max_depth=1, n_estimators=700 ...................
[CV]  max_features=2, max_depth=1, n_estimators=500, score=-0.0793485861626447, total=   0.7s
[CV] max_features=2, max_depth=1, n_estimators=700 ...................
[CV]  max_features=2, max_depth=1, n_estimators=500, score=-0.07067130304605876, total=   0.7s
[CV] max_features=2, max_depth=1, n_estimators=700 ...................


[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.5s


[CV]  max_features=1, max_depth=1, n_estimators=700, score=-0.07149824956775497, total=   0.9s
[CV] max_features=2, max_depth=1, n_estimators=700 ...................
[CV]  max_features=1, max_depth=1, n_estimators=700, score=-0.06634271194433532, total=   1.0s
[CV] max_features=3, max_depth=1, n_estimators=500 ...................
[CV]  max_features=2, max_depth=1, n_estimators=500, score=-0.07314026281396803, total=   0.7s
[CV]  max_features=2, max_depth=1, n_estimators=500, score=-0.06895490128608163, total=   0.7s
[CV] max_features=3, max_depth=1, n_estimators=500 ...................
[CV] max_features=3, max_depth=1, n_estimators=500 ...................
[CV]  max_features=2, max_depth=1, n_estimators=700, score=-0.0649517214280109, total=   1.1s
[CV] max_features=3, max_depth=1, n_estimators=500 ...................


...

[Parallel(n_jobs=-1)]: Done 496 tasks      | elapsed:  1.5min


[CV]  max_features=2, max_depth=9, n_estimators=700, score=-0.01692778894128202, total=   1.7s
[CV] max_features=3, max_depth=9, n_estimators=700 ...................
[CV]  max_features=2, max_depth=9, n_estimators=700, score=-0.011981229164762373, total=   1.8s
[CV] max_features=3, max_depth=9, n_estimators=700 ...................
[CV]  max_features=3, max_depth=9, n_estimators=500, score=-0.014145221558561124, total=   1.5s
[CV] max_features=3, max_depth=9, n_estimators=700 ...................
[CV]  max_features=2, max_depth=9, n_estimators=700, score=-0.015746239762435908, total=   1.7s
[CV] max_features=3, max_depth=9, n_estimators=700 ...................
[CV]  max_features=3, max_depth=9, n_estimators=500, score=-0.01879873210657358, total=   1.5s
[CV] max_features=3, max_depth=9, n_estimators=700 ...................
[CV]  max_features=3, max_depth=9, n_estimators=500, score=-0.016882476796036876, total=   1.5s
[CV] max_features=4, max_depth=9, n_estimators=500 ...................


[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed:  1.7min finished


CPU times: user 2 s, sys: 184 ms, total: 2.18 s
Wall time: 1min 43s


GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=1,
           max_features=2, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=500, n_jobs=1, oob_score=False, random_state=67,
           verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_features': range(1, 7), 'max_depth': range(1, 10), 'n_estimators': [500, 700]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_log_error', verbose=10)

In [44]:
grid_search_tree.best_params_

{'max_depth': 5, 'max_features': 3, 'n_estimators': 700}

In [45]:
np.sqrt(grid_search_tree.best_score_ * -1)

0.12288472290468638